In [1]:
import torch

dbarf = torch.load('data/ibrnet/train/out/ft_dgaussian_019/model/model_001000.pth',map_location=torch.device('cuda:0'))
pixelsplat = torch.load('/mnt/disk1/codes/lihao/mvsplat/outputs/2024-03-22/19-31-03/checkpoints/step_1500.ckpt',map_location=torch.device('cuda:0'))
print(dbarf.keys())
new_model = {}
new_model['pose_learner'] = dbarf['pose_learner']
new_model['gaussian'] = pixelsplat['state_dict']
torch.save(new_model,'model_zoo/pf_mvsplat.pth')

dict_keys(['step', 'pose_learner', 'gaussian', 'pose_optimizer', 'pose_scheduler'])


In [2]:
import torch
model = torch.load('model_zoo/pf_mvsplat.pth')
print(model['gaussian'].keys())

odict_keys(['encoder.backbone.backbone.conv1.weight', 'encoder.backbone.backbone.layer1.0.conv1.weight', 'encoder.backbone.backbone.layer1.0.conv2.weight', 'encoder.backbone.backbone.layer1.1.conv1.weight', 'encoder.backbone.backbone.layer1.1.conv2.weight', 'encoder.backbone.backbone.layer2.0.conv1.weight', 'encoder.backbone.backbone.layer2.0.conv2.weight', 'encoder.backbone.backbone.layer2.0.downsample.0.weight', 'encoder.backbone.backbone.layer2.0.downsample.0.bias', 'encoder.backbone.backbone.layer2.1.conv1.weight', 'encoder.backbone.backbone.layer2.1.conv2.weight', 'encoder.backbone.backbone.layer3.0.conv1.weight', 'encoder.backbone.backbone.layer3.0.conv2.weight', 'encoder.backbone.backbone.layer3.0.downsample.0.weight', 'encoder.backbone.backbone.layer3.0.downsample.0.bias', 'encoder.backbone.backbone.layer3.1.conv1.weight', 'encoder.backbone.backbone.layer3.1.conv2.weight', 'encoder.backbone.backbone.conv2.weight', 'encoder.backbone.backbone.conv2.bias', 'encoder.backbone.transf

In [2]:
import torch
new_ipos = torch.load('data/ibrnet/train/out/pf_mvsplat_ft_019_ipnet_4/model/model_000000.pth')
dbarf = torch.load('data/ibrnet/train/out/ft_dgaussian_019/model/model_001000.pth',map_location=torch.device('cuda:0'))
mvsplat = torch.load('data/ibrnet/train/out/ft_mvsplat_019_wo_depth_view_4/model/model_003000.pth',map_location=torch.device('cuda:0'))

In [3]:
for block in mvsplat['gaussian'].keys():
    print(block)

encoder.backbone.backbone.conv1.weight
encoder.backbone.backbone.layer1.0.conv1.weight
encoder.backbone.backbone.layer1.0.conv2.weight
encoder.backbone.backbone.layer1.1.conv1.weight
encoder.backbone.backbone.layer1.1.conv2.weight
encoder.backbone.backbone.layer2.0.conv1.weight
encoder.backbone.backbone.layer2.0.conv2.weight
encoder.backbone.backbone.layer2.0.downsample.0.weight
encoder.backbone.backbone.layer2.0.downsample.0.bias
encoder.backbone.backbone.layer2.1.conv1.weight
encoder.backbone.backbone.layer2.1.conv2.weight
encoder.backbone.backbone.layer3.0.conv1.weight
encoder.backbone.backbone.layer3.0.conv2.weight
encoder.backbone.backbone.layer3.0.downsample.0.weight
encoder.backbone.backbone.layer3.0.downsample.0.bias
encoder.backbone.backbone.layer3.1.conv1.weight
encoder.backbone.backbone.layer3.1.conv2.weight
encoder.backbone.backbone.conv2.weight
encoder.backbone.backbone.conv2.bias
encoder.backbone.transformer.layers.0.self_attn.q_proj.weight
encoder.backbone.transformer.la

In [3]:
from fnmatch import fnmatch, fnmatchcase
#将高斯权重迁移到mvsplat_iponet
for block in new_ipos['gaussian'].keys():
    if fnmatch(block,'encoder.depth_predictor.pose_head*'):
        block_match = block[24:] 
        new_ipos['gaussian'][block] = dbarf['pose_learner'][block_match]
    if fnmatch(block,'encoder.depth_predictor.update_block_pose*'):
        block_match = block[24:] 
        new_ipos['gaussian'][block] = dbarf['pose_learner'][block_match]
    if fnmatch(block,'encoder.depth_predictor.cnet_pose.upconv1*'):
        block_match = block[24:] 
        new_ipos['gaussian'][block] = dbarf['pose_learner'][block_match]
    if fnmatch(block,'encoder.depth_predictor.corr_refine_net*'):
        new_ipos['gaussian'][block] = mvsplat['gaussian'][block]
    if fnmatch(block,'encoder.depth_predictor.regressor_residual*'):
        new_ipos['gaussian'][block] = mvsplat['gaussian'][block]
    if fnmatch(block,'encoder.depth_predictor.depth_head_lowres*'):
        new_ipos['gaussian'][block] = mvsplat['gaussian'][block]
    if fnmatch(block,'encoder.depth_predictor.upsampler*'):
        new_ipos['gaussian'][block] = mvsplat['gaussian'][block]
    if fnmatch(block, 'encoder.depth_predictor.proj_feature*'):
        new_ipos['gaussian'][block] = mvsplat['gaussian'][block]
    if fnmatch(block,'encoder.depth_predictor.refine_unet*'):
        new_ipos['gaussian'][block] = mvsplat['gaussian'][block]
    if fnmatch(block,'encoder.depth_predictor.to_gaussians*'):    
        new_ipos['gaussian'][block] = mvsplat['gaussian'][block]
    if fnmatch(block,'encoder.depth_predictor.to_disparity*'):
        new_ipos['gaussian'][block] = mvsplat['gaussian'][block]
    if fnmatch(block,'encoder.backbone*'):
        new_ipos['gaussian'][block] = mvsplat['gaussian'][block]

for block in mvsplat['gaussian'].keys():
    new_ipos['gaussian'][block] = mvsplat['gaussian'][block]

new_ipos['pose_learner'] = dbarf['pose_learner']
torch.save(new_ipos,'model_zoo/pf_mvsplat_iponet_1.pth')
# print(fnmatch('foo.txt', '*.txt'))   #True
# print(fnmatch('foo.txt', '?oo.txt')) #True